<a href="https://colab.research.google.com/github/Srujgit/CrimeAnalysis/blob/main/CrimesInBoston.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Crimes in Boston

This project aims to analyse crimes in Boston.

Source: https://www.kaggle.com/datasets/AnalyzeBoston/crimes-in-boston

In [1]:
### Importing basic libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline